In [1]:
def Make_tree2array(names_file):
    names = {}

    for line in names_file:
        if line == "\n":
            #print("empty row")
            continue

        data = [attr.strip() for attr in line.split("|")[0:2]]
        names[data[1]] = int(data[0])

    return names


def Nodes2Array(nodes_file):
    nodes = {}

    for line in nodes_file:
        if line == "\n":
            #print("empty row")
            continue

        data = [attr.strip() for attr in line.split("|")[0:3]]

        nodes[int(data[0])] = [int(data[1]), data[2]]
    return nodes


def Find_Value2Key_in_Names(Key, nodes, names, Taxon):
    if Key in sorted(names):
        Key_in_nodes = names[Key]
        if Key_in_nodes in sorted(nodes):
            if nodes[Key_in_nodes][1] == Taxon:
                #print("FOUND THE KEY")
                return Key
            # ----------------------------------------------------------------------------
            else:
                Find_Value2Key_in_Names(nodes[Key_in_nodes][0], nodes, names, Taxon)
        else:
            print("Key_in_nodes cannot be found.")
    else:
        #print("Key cannot be found.LABEL")
        return "UNKNOWN"
    #print("")



names_file = open("csado7/SILVA132/taxonomy/names.dmp", "r")
nodes_file = open("csado7/SILVA132/taxonomy/nodes.dmp", "r")

names = Make_tree2array(names_file)
nodes = Nodes2Array(nodes_file)


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
import operator
import math

import Bio

import matplotlib.pyplot as plt

seqs = []
groups = []

transformed_group = []

class GroupedByGenus_reads():
    
    def __init__(self,Genus,ListOfGenus):
        self.Genus = Genus
        self.ListOfGenus = ListOfGenus
        self.reads = []
     
    def _add(self,element):
        self.ListOfGenus.append(element)
    
    def _read_list_extend(self, element):
        self.reads+=element
    
            
    def _add(self,element):
        self.ListOfGenus.append(element)

    def fill_reads(self,isFasta=True):
        asd=0
        for temp in self.ListOfGenus:
            #self.reads +=self.create_reads(temp.seq,temp.name)
            asd+=1
            if isFasta:
                self.reads.extend(create_reads_fasta(temp.seq,temp.name,self.Genus))
            else:
                self.reads.extend(create_reads(temp.seq,temp.name,self.Genus))
            
            
            
            if asd%5000==0:
                print(str(asd))
                 

class GroupedByGenus():
    
    def __init__(self,Genus,ListOfGenus):
        self.Genus = Genus
        self.ListOfGenus = ListOfGenus
        
    def _add(self,element):
        self.ListOfGenus.append(element)

count=int(0)
with open("csado7/SILVA132/SILVA/library/silva.fna") as input_handle:
    for record in SeqIO.parse(input_handle, "fasta"):
        count+=1
        
        #curr_genus = record.description.split(' ',1)[1].split(';')[-2]
        #curr_genus = record.description.split(' ',1)[1].split(';')[6]
        curr_genus = str(Find_Value2Key_in_Names(record.description.split(' ',1)[1].split(';')[-2], nodes, names, "genus"))
        #print(record.description.split(' ',1)[1].split(';')[-1])
        
        if type(curr_genus) != str:
            print(count)
            print("Ennyi")
            print(record.description)
            print(curr_genus)
            break
   
        change =0
            
        if count %15000==0:
            print(int(count)/510508*100)

        for i in range(0,len(transformed_group)):
            if transformed_group[i].Genus == curr_genus:
                #transformed_group[i].ListOfGenus.append(record)
                transformed_group[i]._add(record)
                change+=1
                
        if change ==0:
            gp = GroupedByGenus_reads(curr_genus,[])
            gp._add(record)
            #gp.Genus = curr_genus
            #gp.ListOfGenus.append(record)
            transformed_group.append(gp)
        
    
        #if not curr_genus in groups:
        #   groups.append(curr_genus) 
        #seqs.append(record)
        
print(count)
print(str(len(transformed_group)))

AttributeError: 'SeqRecord' object has no attribute 'length'

In [3]:

def create_reads(curr_seq,name,genus,a=30):
        return_string= []
        coverage = [0] * len(curr_seq)
        #--------------------

        c = int((len(curr_seq)+50)/(((len(curr_seq)-200)*a)/150))
        

        #--------------------
        x = len(curr_seq)

        short_reads = 0

        it_end =0

        Length_diff =[]

        while it_end < len(curr_seq):

            if it_end+150 > len(curr_seq):
                if it_end + 50 > len(curr_seq):
                    it_end = len(curr_seq)+1
                    break
                else:
                    read_end =len(curr_seq)
            else:
                read_end = it_end+150


            for delete in range(it_end,read_end):
                coverage[delete] += 1
                
            
            string_out = "@"+name + "|" +genus+"|"+ "S"+str(it_end)  + "E" + str(read_end) + "\n" \
                         + str(curr_seq[it_end:read_end]) + "\n"+ "+" \
            +"\n" +"H" * (read_end-it_end)+"\n"
            
            
            return_string.append(string_out)

            Length_diff.append(read_end-it_end)


            it_end += c
        return return_string
    
def create_reads_fasta(curr_seq,name,genus,a=30):
        return_string= []
        coverage = [0] * len(curr_seq)
        #--------------------

        c = int((len(curr_seq)+50)/(((len(curr_seq)-200)*a)/150))
        

        #--------------------
        x = len(curr_seq)

        short_reads = 0

        it_end =0

        Length_diff =[]

        while it_end < len(curr_seq):

            if it_end+150 > len(curr_seq):
                if it_end + 50 > len(curr_seq):
                    it_end = len(curr_seq)+1
                    break
                else:
                    read_end =len(curr_seq)
            else:
                read_end = it_end+150


            for delete in range(it_end,read_end):
                coverage[delete] += 1
                
            
            string_out = ">"+name + "|" +genus+"|"+ "S"+str(it_end)  + "E" + str(read_end) + "\n" \
                         + str(curr_seq[it_end:read_end]) + "\n"
            
            
            return_string.append(string_out)

            Length_diff.append(read_end-it_end)


            it_end += c
        return return_string

    
def print_(seq):
    print(seq)



In [4]:
import Bio


"""
for temp in groups:
    new_type = GroupedByGenus_reads(temp)
    transformed_group.append(new_type)
"""
#-----------------

#-----------------
t=transformed_group[0].ListOfGenus[0].seq

x_ =0

for el in transformed_group:
    x_+=1
    if el.Genus == "UNKNOWN":
        print("c:")
        continue
    
    #ez csak visszajelzés h hol jár
    if x_%50==0:
        print(str(len(el.ListOfGenus))+" , "+el.Genus+":  " + str(x_/len(transformed_group)*100)+"%-nál jár a beolvasás.")
    
    el.fill_reads()

    

_asd=0
for s in transformed_group:
#for s in range(0,1):
    path= "Genus_FASTA_Format_20221008/"+s.Genus+"_reads.fasta"
    File_Genus = open(path, 'w')
    
    for r in s.reads:    
        File_Genus.write(r)
        _asd+=1
    File_Genus.close()
    del s
print(str(_asd))




5000
5000
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
5000
5000
10000
15000
5000
c:
98 , Leifsonia:  0.6005284650492433%-nál jár a beolvasás.
5000
802 , Citrobacter:  1.2010569300984866%-nál jár a beolvasás.
475 , Candidatus Phytoplasma:  1.80158539514773%-nál jár a beolvasás.
1 , Amischotolype:  2.402113860196973%-nál jár a beolvasás.
113 , Muricauda:  3.0026423252462164%-nál jár a beolvasás.
332 , Micrococcus:  3.60317079029546%-nál jár a beolvasás.
2344 , Flavobacterium:  4.203699255344703%-nál jár a beolvasás.
15 , Lysinimonas:  4.804227720393946%-nál jár a beolvasás.
5000
319 , Mucilaginibacter:  5.4047561854431905%-nál jár a beolvasás.
118 , Arabidopsis:  6.005284650492433%-nál jár a beolvasás.
3 , Thermocatellispora:  6.605813115541677%-nál jár a beolvasás.
12 , Stichococcus:  7.20634158059092%-nál jár a beolvasás.
86 , Dactylococcopsis PCC-8305:  7.806870

In [5]:
#generál Genuonként fájlokat 

#fasta files

print(len(transformed_group))
"""
_asd=0
for s in transformed_group:
#for s in range(0,1):
    path= "10622_Genus_reads__fasta_v2/"+s.Genus+"_reads.fasta"
    File_Genus = open(path, 'w')
    
    for r in s.reads_fasta:    
        File_Genus.write(r)
        _asd+=1
    File_Genus.close()
print(str(_asd))

"""



8326


'\n_asd=0\nfor s in transformed_group:\n#for s in range(0,1):\n    path= "10622_Genus_reads__fasta_v2/"+s.Genus+"_reads.fasta"\n    File_Genus = open(path, \'w\')\n    \n    for r in s.reads_fasta:    \n        File_Genus.write(r)\n        _asd+=1\n    File_Genus.close()\nprint(str(_asd))\n\n'

In [15]:
print(len(transformed_group[].reads))

344
